# MOUSENEU_LP_N2V_1: Noise2Void 2D on Mouse Neuriter Dataset

Dataset is composed one video of mouse neurites for training and one for validation.
Videos have size (6500, 490, 490) and are acquired using 66mW power, 30hz frame rate and 40um depth.

In [ ]:
#!pip install microssim


In [ ]:
import tifffile
from pathlib import Path
from microssim import MicroSSIM, micro_structural_similarity
from skimage.metrics import structural_similarity
from careamics.utils.metrics import psnr, scale_invariant_psnr
import numpy as np
import pandas as pd

DATASET_PATH = Path("/localscratch/calcium_imaging_dataset/calcium_imaging/MOUSENEU_LP/")
PREDICTIONS_PATH = Path("output")
MODELS_TO_COMPARE = ['MOUSENEU_LP_N2N_1', 'MOUSENEU_LP_N2V_1', 'MOUSENEU_LP_DEEPCAD_1']
SPLIT_TO_COMPARE = ['train', 'val']

In [ ]:
results = []


for split in SPLIT_TO_COMPARE:
    input_paths = list(DATASET_PATH.joinpath(f"{split}").glob("*.tif"))
    for input_path in input_paths:
        filename = str(input_path.name)
        gt_path = DATASET_PATH.joinpath(f"{split}_gt").joinpath(filename.replace("lowSNR", "highSNR"))
        
        gt_image = tifffile.imread(gt_path).astype(np.uint16)
        predictions_paths = [(PREDICTIONS_PATH.joinpath(model).joinpath(f"{split}").joinpath(filename), model) for model in MODELS_TO_COMPARE]

        for target_path, target_name in [(input_path, "lowSNR")] + predictions_paths:
            target_image = tifffile.imread(target_path).astype(np.uint16)
            ms = MicroSSIM()
            # Fit microSSIM parameters with 1/10 of the frame to avoid OOM
            ms.fit(gt=gt_image[::10], pred=target_image[::10])

            for f, (gt_frame, target_frame) in enumerate(zip(gt_image, target_image)):
                res = dict()
                res["frame"] = f
                res["target_name"] = target_name
                res["split"] = split
                res["gt_fp"] = gt_path.name
                res["target_fp"] = target_path.name
                res["ssim"] = structural_similarity(gt_frame, target_frame, data_range=65535)
                res["microssim"] = ms.score(gt=gt_frame, pred=target_frame)
                res["psnr"] = psnr(gt=gt_frame, pred=target_frame, data_range=65535)
                res["si_psnr"] = scale_invariant_psnr(gt=gt_frame, pred=target_frame)
                results.append(res)


In [ ]:
pd_results = pd.DataFrame.from_dict(results)
pd_results.to_csv("results/MOUSENEU_LP.csv")

In [ ]:
means = pd_results.groupby(["split","target_name", "gt_fp", "target_fp"]).mean().drop(columns=["frame"])
std = pd_results.groupby(["split","target_name", "gt_fp", "target_fp"]).std().drop(columns=["frame"])
formatted_df = means.copy()
for col in means.columns:
    formatted_df[col] = means[col].map('{:.3f}'.format) + " ± " + std[col].map('{:.3f}'.format)



In [ ]:
formatted_df.to_csv("results/MOUSENEU_LP_formatted.csv")

In [ ]:
import pandas as pd
formatted_df = pd.read_csv("results/MOUSENEU_LP_formatted.csv", index_col=0)
with open("results/MOUSENEU_LP.md", "w") as f:
    f.write(formatted_df.to_markdown())